In [145]:
import requests
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as nd
import tensorflow as tf
import tensorflow_hub as hub
import imageio
from google_images_download import google_images_download
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import PIL.Image
import PIL.ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.filters import gaussian
from skimage.measure import ransac
from skimage.transform import AffineTransform
from skimage import color, io
from skimage.exposure import rescale_intensity


In [138]:
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1")
height, width = hub.get_expected_image_size(module)
height, width

(224, 224)

In [139]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal national park", "limit": 20,
             "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal national park
Evaluating...
Starting Download...
Image URL: https://i.ytimg.com/vi/P8NJa_YoRxk/maxresdefault.jpg
Completed Image ====> 1.maxresdefault.jpg
Image URL: https://k6u8v6y8.stackpathcdn.com/blog/wp-content/uploads/2017/06/Royal-Bengal-Tiger.jpg
Completed Image ====> 2.Royal-Bengal-Tiger.jpg
Image URL: https://www.corbettnationalpark.in/blog/wp-content/uploads/2015/08/cropped-13625772024_1fd7467d29_k1.jpg
Completed Image ====> 3.cropped-13625772024_1fd7467d29_k1.jpg
Image URL: https://www.nps.gov/arch/learn/nature/images/ARK_6.jpg?maxwidth=1200&maxheight=1200&autorotate=false
Completed Image ====> 4.ARK_6.jpg
Image URL: https://npca.s3.amazonaws.com/images/8135/2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg?1445969501
Completed Image ====> 5.2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg
Image URL: https://k6u8v6y8.stackpathcdn.com/blog/wp-content/uploads/2014/04/national-parks-and-wildlife-sanctuaries-in-india.png
Completed Image ====> 6.

In [140]:
image_list = absolute_image_paths[0]['animal national park']
image_list

['C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\1.maxresdefault.jpg',
 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\2.Royal-Bengal-Tiger.jpg',
 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\3.cropped-13625772024_1fd7467d29_k1.jpg',
 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\4.ARK_6.jpg',
 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\5.2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg',
 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\6.national-parks-and-wildlife-sanctuaries-in-india.png',
 'C:\\Users\\lilyx\\DS-Unit-4-

In [73]:
def resize_image(filename, new_width=256, new_height=256):#the delf model was trained on images 256x256 the input of our model also should be 256x256
  pil_image = Image.open(filename)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert('RGB')
  pil_image_rgb.save(filename, format='JPEG', quality=90)

for i in image_list:
     resize_image(i)

In [134]:
import os
pathimg = os.path.abspath('C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark')
print(pathimg)

C:\Users\lilyx\DS-Unit-4-Sprint-3-Deep-Learning\module2-convolutional-neural-networks\downloads\animalnationalpark


In [135]:
for filename in os.listdir('C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark'):
    if filename.endswith('.jpg'):
        print(filename)

1.maxresdefault.jpg
10.moose-moosecalf-1280x853.jpg
10.zebras-in-kidepo-750x450.jpg
11.addo-elephant-national-park-elephants-590x390.jpg
11.zebras-in-kidepo-750x450.jpg
12.lion-520x400.jpg
12.nameri-national-park-is-famous-for-which-animal.jpg
13.lion-520x400.jpg
13.Nairobi_National_Park%2C_Kenya_%2832570316676%29.jpg
14.lion-520x400.jpg
14.nairobi-national-park.jpg
14.Nairobi_National_Park%2C_Kenya_%2832570316676%29.jpg
14.Tiger_Kanha_National_Park.jpg
15.Bandhavgarh.jpg
15.lion-520x400.jpg
15.nairobi-national-park.jpg
15.Nairobi_National_Park%2C_Kenya_%2832570316676%29.jpg
15.Tiger_Kanha_National_Park.jpg
16.Bandhavgarh.jpg
16.Dudhwa-National-Park.jpg
16.Sariska-national-Park.jpg
16.Tiger_Kanha_National_Park.jpg
17.Bandhavgarh.jpg
17.Dudhwa-National-Park.jpg
17.Sariska-national-Park.jpg
18.2-day-kruger-slider1.jpg
18.Dudhwa-National-Park.jpg
18.Grizzly-DanRafla.jpg
18.Sariska-national-Park.jpg
19.2-day-kruger-slider1.jpg
19.Sariska-national-Park.jpg
19.Van-Vihar-National-Parkbhopal.j

In [77]:
for filename in os.listdir('C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark'):
    if filename.endswith('.jpg'):
        resize_image(filename)

FileNotFoundError: [Errno 2] No such file or directory: '1.maxresdefault.jpg'

In [61]:
import os
import glob

for filepath in glob.glob(os.path.join('C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark', '*.jpg')):
    with open(filepath, encoding="utf8") as f:
        content = f.read()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [66]:
with open(r'C:\Users\lilyx\DS-Unit-4-Sprint-3-Deep-Learning\module2-convolutional-neural-networks\downloads\animalnationalpark', encoding='utf-8') as f:
    contents = f.read()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark'

In [39]:
resize_image(image_list[0])
resize_image(image_list[1])
        

In [78]:
for i in range(len(image_list)):
    resize_image(image_list[i])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\1.maxresdefault.jpg'

In [80]:
image_list[3]

'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\4.cropped-13625772024_1fd7467d29_k1.jpg'

In [84]:
resize_image(image_list[0])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\1.maxresdefault.jpg'

In [85]:
for i in image_list:
     resize_image(i)
# len(image_list)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\1.maxresdefault.jpg'

In [86]:
images = [resize_image(image) for image in image_list]
images

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\1.maxresdefault.jpg'

In [70]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks': ['.ipynb_checkpoints', '442_assignment.ipynb', 'downloads', 'example0.jpg', 'example1.jpg', 'LS_DS_442_Convolutional_Neural_Networks.ipynb', 'tower0.jpg', 'tower1.jpg']


In [103]:
from PIL import Image 
  
filename = "C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark\\1.maxresdefault.jpg"
with Image.open(filename) as image: 
    width, height = image.size
    
width

256

In [101]:
from PIL import Image 
  
filename = filename
with Image.open(filename) as image: 
    width, height = image.size 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\4.cropped-13625772024_1fd7467d29_k1.jpg'

In [115]:
from PIL import Image 
  
filename = "C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark"
#with Image.open(filename) as image: 
files = os.listdir(filename)
for i in files:
    resize_image(i)
    width, height = image.size
    print(width)



FileNotFoundError: [Errno 2] No such file or directory: '1.maxresdefault.jpg'

In [129]:
import os
THIS_FOLDER = os.path.dirname(os.path.abspath('C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark'))
my_file = os.path.join(THIS_FOLDER, 'animalnationalpark\\1.maxresdefault.jpg')
#resize_image(my_file)
for i in os.listdir(my_file):
    print(i)

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animalnationalpark\\1.maxresdefault.jpg'

In [146]:
def show_images(image_list):
    for i in image_list:
        resize_image(i)
show_images(image_list)

OSError: cannot identify image file 'C:\\Users\\lilyx\\DS-Unit-4-Sprint-3-Deep-Learning\\module2-convolutional-neural-networks\\downloads\\animal national park\\18.Dudhwa-National-Park.jpg'